In [23]:
%load_ext autoreload
%autoreload 2

from metal.mmtl.task import Task
from metal.mmtl.scorer import Scorer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# Deprecated
# #########################
# # Create Ines's model 
# #########################
# import os 

# import numpy as np
# import pandas as pd
# import torch
# import torch.nn as nn
# import torch.functional as F
# from tqdm import tqdm
# from pytorch_pretrained_bert import BertTokenizer, BertModel

# from metal.mmtl.dataset import BERTDataset
# from metal.end_model import EndModel

# batch_size = 32
# max_len = 200
# weight_decay = 0.01
# epochs = 1
# lr = 0.001

# model = 'bert-base-uncased' # also try bert-base-multilingual-cased (recommended)
# src_path = os.path.join(os.environ['GLUEDATA'], 'QNLI/{}.tsv')
# dataloaders = {}
# for split in ['train', 'test', 'dev']: #, 'train', 'test']:
#     label_idx = 3 if split in ['train', 'dev'] else -1
#     dataset = BERTDataset(
#         src_path.format("dev"),
#         sent1_idx=1,
#         sent2_idx=2,
#         label_idx=label_idx,
#         skip_rows=400,
#         label_fn=lambda label: 1 if label=='entailment' else 2 
#     )
#     dataloaders[split] = dataset.get_dataloader(max_len=max_len, batch_size=batch_size)
    
# class BertEncoder(nn.Module):
#     def __init__(self):
#         super(BertEncoder, self).__init__()
#         self.bert_model = BertModel.from_pretrained('bert-base-uncased')
#         for param in self.bert_model.parameters():
#             param.requires_grad = False
        
#     def forward(self, data):
#         tokens, segments, masks = data
#         # TODO: check if we should return all layers or just last hidden representation 
#         _, hidden_layer = self.bert_model(input_ids=tokens, token_type_ids=segments, attention_mask=masks)
#         return hidden_layer
    
# encoder_module = BertEncoder()
# end_model = EndModel(
#     [768, 2],  # TODO: remove bias
#     input_module=encoder_module,
#     seed=123,
#     skip_head=False,
#     input_relu=False,
#     input_batchnorm=False,
#     verbose=False,
#     device=torch.device('cuda'),
# )

# end_model.train_model(
#     train_data=dataloaders['dev'],
#     valid_data=dataloaders['dev'],
#     l2=weight_decay,
#     lr=lr,
#     n_epochs=epochs,
#     verbose=True,
#     checkpoint=False,
#     log_unit='epochs', 
#     log_train_every=1,
#     log_valid_every=1,
#     progress_bar=True,
# )
    

In [25]:
# Deprecated
# # Test 
# def custom_eval_function(Y, Y_pred, probs=None):
#     print("Running custom_eval_function")
#     return {"custom_metric" : 0}

# # Create a scorer (standard_metrics are broken)
# dummy_scorer = Scorer(standard_metrics=[], custom_metric_fns=[custom_eval_function])

# # Create task with scorer
# data_loaders = [dataloaders[x] for x in ["train", "test", "dev"]]
# foo_task = Task(name="foo_task", 
#                 input_module=encoder_module,
#                 head_module=end_model,
#                 data_loaders=data_loaders, scorers=[scorer])

# # Call scorer on model / task / etc
# dummy_scorer(foo_task, end_model, data_loaders[-1], split_name="test_scorer")

In [26]:
# Deprecated
# # Test standard loss function

# # Create a scorer (standard_metrics are broken)
# dummy_loss_scorer = Scorer(standard_metrics=["f1"])

# # Create task with scorer
# data_loaders = [dataloaders[x] for x in ["train", "test", "dev"]]
# foo_task = Task(name="foo_task", 
#                 input_module=encoder_module,
#                 head_module=end_model,
#                 data_loaders=data_loaders, scorers=[dummy_loss_scorer])

# # Call scorer on model / task / etc
# dummy_loss_scorer(foo_task, end_model, data_loaders[-1], split_name="test_scorer")

In [27]:
# Test head_output optimization

In [29]:
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

def make_dataloader(n):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=4)
    return data_loader

import torch.nn as nn
from metal.mmtl.task import Task

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

scorer = Scorer()
foo = Task("foo_task", [make_dataloader(5000), make_dataloader(500), make_dataloader(500)], foo_input, foo_head, scorers=[scorer])
bar = Task("bar_task", [make_dataloader(2000), make_dataloader(500), make_dataloader(500)], bar_input, bar_head, scorers=[scorer])
tasks = [foo, bar]

from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks)
trainer = MultitaskTrainer()
trainer.train_model(model, tasks, n_epochs=1)


Network architecture:
MetalModel(
  (input_modules): ModuleDict(
    (bar_task): Linear(in_features=2, out_features=10, bias=True)
    (foo_task): Linear(in_features=2, out_features=10, bias=True)
  )
  (head_modules): ModuleDict(
    (bar_task): Linear(in_features=10, out_features=2, bias=True)
    (foo_task): Linear(in_features=10, out_features=2, bias=True)
  )
  (task_paths): ModuleDict(
    (bar_task): Sequential(
      (0): Linear(in_features=2, out_features=10, bias=True)
      (1): Linear(in_features=10, out_features=2, bias=True)
    )
    (foo_task): Sequential(
      (0): Linear(in_features=2, out_features=10, bias=True)
      (1): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

Batch 0 of 100


Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f0dbd2ea570>
Traceback (most recent call last):
  File "/dfs/scratch0/vschen/venv-mmtl/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 226, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


KeyError: 'device'